In [20]:
import numpy as np
from sklearn.datasets.mldata import fetch_mldata
import tempfile
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans 
from sklearn import cluster
from sklearn.mixture import GMM  # contiene los modelos de mezclas 
from sklearn.cross_validation import StratifiedKFold # Dividir por k-folds
from sklearn.metrics import classification_report
from sklearn import cross_validation
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [16]:
test_data_home = tempfile.mkdtemp()
leuk = fetch_mldata('leukemia', transpose_data=True, data_home=test_data_home)

X = leuk['data']

y = leuk['target']

In [4]:
print(np.unique(y))

[-1  1]


# Preprocesamiento - Remoción de la media

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

def remocion_train(X):
    X = X - X.mean(axis=0)
    X = X/X.std(axis=0)
    
    return X

def remocion_test(X_test):
    media = X_train.mean(axis=0)
    desviacion = X_train.std(axis=0)
    
    X_test = X_test - media
    X_test = X_test/desviacion
    
    return X_test

x_train_norm = remocion_train(X_train)
x_test_norm = remocion_test(X_test)

# Regresión Logística

In [9]:
# Regresión logística
clasificador = LogisticRegression(C=1000.0, random_state=0)
clasificador.fit(x_train_norm, y_train)
y_pred = clasificador.predict(x_test_norm)


iguales = (y_test == y_pred).sum()
n = len(y_pred)
porcentaje = (iguales / n)*100
print('numero de muestras iguales fueron %d / %d'%( iguales, n))
print('Porcentaje es: %f' %(porcentaje))

numero de muestras iguales fueron 21 / 22
Porcentaje es: 95.454545


In [10]:
nombres_clases = ['Primera Clase', 'Segunda Clase']
print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de entranamiento')
print(classification_report(y_train, clasificador.predict(X_train), target_names = nombres_clases))
print('\n'+'*'*60)

print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de validacion')
print(classification_report(y_test, y_pred, target_names = nombres_clases))
print('\n'+'*'*60)


************************************************************
Desempeño del clasificador sobre el conjunto de entranamiento
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        16
Segunda Clase       1.00      1.00      1.00        34

  avg / total       1.00      1.00      1.00        50


************************************************************

************************************************************
Desempeño del clasificador sobre el conjunto de validacion
               precision    recall  f1-score   support

Primera Clase       0.90      1.00      0.95         9
Segunda Clase       1.00      0.92      0.96        13

  avg / total       0.96      0.95      0.95        22


************************************************************


# Bayes Naive

In [11]:
clasificador = GaussianNB()
clasificador.fit(X_train, y_train)
y_pred = clasificador.predict(X_test)

iguales = (y_test == y_pred).sum()
n = len(y_pred)
porcentaje = (iguales / n)*100
print('numero de muestras iguales fueron %d / %d'%( iguales, n))
print('Porcentaje es: %f' %(porcentaje))

numero de muestras iguales fueron 22 / 22
Porcentaje es: 100.000000


In [12]:
nombres_clases = ['Primera Clase', 'Segunda Clase']
print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de entranamiento')
print(classification_report(y_train, clasificador.predict(X_train), target_names = nombres_clases))
print('\n'+'*'*60)

print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de validacion')
print(classification_report(y_test, y_pred, target_names = nombres_clases))
print('\n'+'*'*60)


************************************************************
Desempeño del clasificador sobre el conjunto de entranamiento
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        16
Segunda Clase       1.00      1.00      1.00        34

  avg / total       1.00      1.00      1.00        50


************************************************************

************************************************************
Desempeño del clasificador sobre el conjunto de validacion
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00         9
Segunda Clase       1.00      1.00      1.00        13

  avg / total       1.00      1.00      1.00        22


************************************************************


# NO SUPERVISADO

# Clúster

In [85]:
num_cluster = 2

modelo_kmeans = KMeans(init = 'k-means++', n_clusters = num_cluster, n_init = 10) # init establece el método de localización inicial de los centroides
modelo_kmeans.fit(X)
centroides = modelo_kmeans.cluster_centers_
labels = modelo_kmeans.labels_

print('Clase 1: %d' %(sum(labels == 1)))
print('Clase -1: %d' %(sum(labels == 0)))

labels_con_clase = list()
for clase in labels:
    dato = 1 if clase == 1 else -1
    labels_con_clase.append(dato)

print('Procentaje iguales:')
print((labels_con_clase == y).sum()/len(y)*100)

Clase 1: 42
Clase -1: 30
Procentaje iguales:
54.1666666667


# Modelo GMM

In [5]:
indices = StratifiedKFold(y, n_folds=4)

train_index, test_index = next(iter(indices))

y = np.where(y==-1,0, y)

X_train = X[train_index]
y_train = y[train_index]

X_test = X[test_index]
y_test = y[test_index]


numero_clases = len(np.unique(y_train))

clasificador = GMM(n_components = numero_clases, covariance_type = 'full', init_params='w', n_iter=20)
clasificador.means_ = np.array([X_train[y_train==i].mean(axis=0) for i in range(numero_clases)])


clasificador.fit(X_train)


y_train_predict = clasificador.predict(X_train)
acc_train = np.mean(y_train_predict.ravel() == y_train.ravel())*100
print('Porcentaje iguales entrenamiento: %d' %acc_train)

y_test_predict = clasificador.predict(X_test)
acc_test = np.mean(y_test_predict.ravel() == y_test.ravel())*100
print('Porcentaje iguales test: %d' %acc_test)

C:\Users\s107e21\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:52: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\s107e21\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The functon distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\s107e21\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\s107e21\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarni

Porcentaje iguales entrenamiento: 100


C:\Users\s107e21\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)


Porcentaje iguales test: 100


# Arboles de decision

In [18]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.25, random_state=5)

clase0 = np.array(X[y==-1])
clase1 = np.array(X[y==1])

parametros = {'random_state':0, 'max_depth':4}


clasificador = DecisionTreeClassifier(**parametros)

clasificador.fit(X_train, y_train)

y_test_pred = clasificador.predict(X_test)


nombres_clases = ['Primera Clase', 'Segunda clase']
print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de entranamiento')
print(classification_report(y_train, clasificador.predict(X_train), target_names = nombres_clases))
print('\n'+'*'*60)

print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de validacion')
print(classification_report(y_test, y_test_pred, target_names = nombres_clases))
print('\n'+'*'*60)



************************************************************
Desempeño del clasificador sobre el conjunto de entranamiento
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        19
Segunda clase       1.00      1.00      1.00        35

  avg / total       1.00      1.00      1.00        54


************************************************************

************************************************************
Desempeño del clasificador sobre el conjunto de validacion
               precision    recall  f1-score   support

Primera Clase       1.00      0.83      0.91         6
Segunda clase       0.92      1.00      0.96        12

  avg / total       0.95      0.94      0.94        18


************************************************************


# Bosques Aleatorios

In [22]:
tipo_clasificador = 'ba'

parametros = { 'n_estimators':100, 'max_depth': 4, 'random_state':0}

if tipo_clasificador == 'ba':
    # Bosque aleatorio
    clasificador = RandomForestClassifier(**parametros)
else:
    # Bosque extra aleatorio
    clasificador = ExtraTreesClassifier(**parametros)

# Entrenamos el clasificador
clasificador.fit(X_train, y_train)

#validamos el clasificador
y_test_pred = clasificador.predict(X_test)


nombres_clases = ['Primera Clase', 'Segunda clase']
print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de entranamiento')
print(classification_report(y_train, clasificador.predict(X_train), target_names = nombres_clases))
print('\n'+'*'*60)

print('\n'+'*'*60)
print('Desempeño del clasificador sobre el conjunto de validacion')
print(classification_report(y_test, y_test_pred, target_names = nombres_clases))
print('\n'+'*'*60)



************************************************************
Desempeño del clasificador sobre el conjunto de entranamiento
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00        19
Segunda clase       1.00      1.00      1.00        35

  avg / total       1.00      1.00      1.00        54


************************************************************

************************************************************
Desempeño del clasificador sobre el conjunto de validacion
               precision    recall  f1-score   support

Primera Clase       1.00      1.00      1.00         6
Segunda clase       1.00      1.00      1.00        12

  avg / total       1.00      1.00      1.00        18


************************************************************
